## Grundlagen Maschineller Lernverfahren | ML_INF19A | 2021
**Datum: 26.10.2021**

# Neuronale Netzwerke
Ressourcen: https://www.tensorflow.org/api_docs/python/tf

## Lade Daten

In [ ]:
# Lade Bibliothek
from sklearn.datasets import load_iris

In [ ]:
# Lade Datensatz
data = load_iris()

In [ ]:
# Daten in Eingabe (X) und Layer (y) sortieren und "lokal" abspeichern
X = data.data
y = data.target

In [ ]:
# Blick in die Datenwerfen
print(X)

In [ ]:
# Blick in die Labels werfen
print(y)

In [ ]:
# Dimensionen prüfen
print(X.shape)
print(y.shape)

## Daten vorverarbeiten

In [ ]:
# Lade Bibliothek
from sklearn.preprocessing import normalize # Normalisiere die Daten

In [ ]:
# Führe Vorverarbeitung durch
X_norm = normalize(X) # NUR Eingabedaten, nicht die Label!

In [ ]:
# Zeige die transformierten Daten an
X_norm

## Trenne in Daten für Training und Test

In [ ]:
# Lade Bibliothek
from sklearn.model_selection import train_test_split

In [ ]:
# Führe Trennung durch
X_train, X_test, y_train, y_test = train_test_split(X_norm,y,test_size=0.2)

In [ ]:
# Prüfe Dimensionen
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

## Neuronales Netz erstellen

In [ ]:
# Lade Bibliotheken
import tensorflow as tf

from tensorflow.keras.models import Sequential # Modelinstanz
from tensorflow.keras.layers import Dense # Einzelne Layer
from tensorflow.keras import Input # Spezieller Eingabelayer

import numpy as np

In [ ]:
# Details über Eingabeformat
X_norm.shape #(150,4) -> 150 Zeilen, 4 Features

In [ ]:
# Prüfe wieviele Label/Klassen in der Aufgabe vorkommen 
num_classes = len(np.unique(y_train)) # -> bestimmt die Anzahl der Knoten im Ausgabelayer

# Prüfe ob für Training und Test gleich viele Klassen vorhanden sind
assert len(np.unique(y_train)) == len(np.unique(y_test))

In [ ]:
## Setup Hyperparameter des Neuronalen Netzwerkes

# Anzahl der Knoten im Eingabelayer
input_size = X_norm.shape[1:] # Nur Anzahl der Features relevant (150 stellt Batch-Dimension dar)

# Anzahl der Knoten im Ausgabelayer
output_size = num_classes # Speichere Anzahl der verschiedenen Klassen

# Größe des Batches beim Training
batch_size = 10

# Anzahl der Epcohen beim Training
epochs = 20

In [ ]:
# Erstelle eine Instanz des Sequential Modells
model = Sequential()

In [ ]:
#model

In [ ]:
# Definition des einfachen Modells (Struktur: Input -> Hidden -> Output)

# Eingabe
model.add(Input(shape=X_norm.shape[1:], name="Eingabe")) # Ohne Batch Dimension, nur "X-Achsen" Dimension

# Hidden Layer (3x)
model.add(Dense(1000, activation="relu", name="Hidden1"))
model.add(Dense(500, activation="relu", name="Hidden2"))
model.add(Dense(300, activation="relu", name="Hidden3"))

# Ausgabe
model.add(Dense(3, activation="softmax", name="Ausgabe")) # Hinweis: Bei mehreren Klassen nimmt man in der Regel die Softmax Aktivierungsfunktion (weitere Infos: vgl. Literatur)

In [ ]:
# "Baue" das Modell zusammen
model.compile(loss='categorical_crossentropy', metrics='accuracy') # Angeben von diversen Hyperparametern (vgl. Doku für mehr Informationen)

## Modell Zusammenfassung

In [ ]:
# Modell anzeigen
model.summary() 

In [ ]:
## Analyse der Parameter

#Anzahl Knoten pro Layer:
#------------------------
#Input   = 4    Knoten
#Hidden1 = 1000 Knoten
#Hidden2 = 500  Knoten
#Hidden3 = 300  Knoten
#Ausgabe = 1000 Knoten

#Allgemein gilt:

# Knoten(L) * #Knoten(L+1) + #Knoten(L+1)
# |----------------------|   |----------|
#        Parameter               Bias

#Berechnung Input->Hidden1:
#4 * 1000 + 1000 = 5000

#Berechnung Hidden1->Hidden2:
#1000 * 500 + 500 = 500500

#Berechnung Hidden2->Hidden3:
#500 * 300 + 300 = 150300

#Berechnung Hidden3->Ausgabe:
#300 * 3 + 3 = 903

Hinweis:

* Das Modell hat 3 Knoten in der Ausgabeschicht
* Die Labels sind aber nur als Zahl (0,1,2) verfügbar.

Die Labels müssen nun also noch in eine für das Neuronale Netzwerk passende Form gebracht werden.



## Vorbereiten der Labels (OneHot Encoding)

In [ ]:
# Prüfe ursprüngliche Label
print(y_train)
print(y_train.shape) # 120x1 Vektor

Wir benötigen aber eine Repräsentation wie folgt:

> 0 -> [1 0 0]

> 1 -> [0 1 0]

> 2 -> [0 0 1]


Lösung: OneHot Vektor

In [ ]:
# Lade Bibliothek
from tensorflow.keras.utils import to_categorical # OneHot-Encoding (vgl. Doku: https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical)

In [ ]:
# Umwandlung in One-Hot Vektor (vgl. Dokumentation "to_categorical")
y_train_1hot = to_categorical(y_train, num_classes)
y_test_1hot = to_categorical(y_test, num_classes)

In [ ]:
# Prüfen des Ergebnisses
print(y_train[:10])
print(y_train_1hot[:10])

## Modell trainieren

In [ ]:
# Training über die bekannte Funktion "fit" unter Angabe von Hyperparametern (vgl. Dokumentation: https://www.tensorflow.org/api_docs/python/tf/keras/Sequential#fit)
model.fit(X_train, y_train_1hot, batch_size=batch_size, epochs=epochs, verbose=1)

## Modell verifizieren

In [ ]:
# Evaluiere das Modell auf Basis der Validierungsdatenmenge
model.evaluate(X_test, y_test_1hot)

## Alternative Modelldefinition

In [ ]:
# Das gleiche Modell, nur anders erstellt.
# Hinweis: Diese Variante ist viel flexibler zum Erstellen von Modellen sobald die Netzwerke komplizierter werden.

# Definiere die "Architektur" (nur Layerverbindungen untereinander, kein Modell!)
inputs = Input(shape=input_size, name="Eingabe")
hidden_1 = Dense(1000,  activation='relu', name="Hidden1")(inputs)
hidden_2 = Dense(500,  activation='relu', name="Hidden2")(hidden_1)
hidden_3 = Dense(300,  activation='relu', name="Hidden3")(hidden_2)
outputs = Dense(3, activation='softmax', name="Ausgabe")(hidden_3)

# Definition des finalen Modells
model_2 = tf.keras.Model(inputs=inputs, outputs=outputs)

# "Baue" Modell
model_2.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='sgd') # Explizite Angabe des Optimierers

# Zeige Modellstruktur an
print(model_2.summary())

In [ ]:
# Training über die bekannte Funktion "fit" unter Angabe von Hyperparametern (vgl. Dokumentation)
model_2.fit(X_train, y_train_1hot, batch_size=batch_size, epochs=epochs, verbose=1)

In [ ]:
# Evaluiere das Modell auf Basis der Validierungsdatenmenge
model_2.evaluate(X_test, y_test_1hot)

## Model speichern und laden

In [ ]:
# Pfad zum Speichern angeben
path = "mein_modell"

# Speichere das Modell in den genannten Pfad
model.save(path)

In [ ]:
# Gespeichertes Model laden
model_restored = tf.keras.models.load_model(path)

In [ ]:
# Vergleichen der Modelle
model_restored.summary()
model.summary()